## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-16-23-51-16 +0000,wapo,Trump ups the ante in Venezuela standoff,https://www.washingtonpost.com/world/2025/11/1...
1,2025-11-16-23-48-45 +0000,nypost,Ukraine hits major Russian oil refinery and si...,https://nypost.com/2025/11/16/world-news/ukrai...
2,2025-11-16-23-47-29 +0000,nyt,"Latest U.S. Strike on Boat in Pacific Kills 3,...",https://www.nytimes.com/2025/11/16/us/politics...
3,2025-11-16-23-46-00 +0000,wsj,Gold Rises on Likely Technical Recovery,https://www.wsj.com/finance/commodities-future...
4,2025-11-16-23-42-32 +0000,nypost,Bearded cane-toting man shoots guy on NYC street,https://nypost.com/2025/11/16/us-news/bearded-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,17
197,epstein,9
307,government,8
169,will,7
495,dead,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
47,2025-11-16-20-26-00 +0000,wsj,President Trump is making a last-ditch push to...,https://www.wsj.com/politics/policy/trump-play...,55
27,2025-11-16-21-32-28 +0000,bbc,Marjorie Taylor Greene doubles down on Epstein...,https://www.bbc.com/news/articles/cdeg8z0j2j2o...,51
122,2025-11-16-08-12-45 +0000,bbc,Feud erupts between Trump and ally Marjorie Ta...,https://www.bbc.com/news/articles/c9wv4dx05q5o...,50
95,2025-11-16-14-50-10 +0000,nypost,Marjorie Taylor Greene insists she still suppo...,https://nypost.com/2025/11/16/us-news/marjorie...,49
88,2025-11-16-16-58-43 +0000,nypost,MTG suggests Israel or ‘other foreign gov’t’ m...,https://nypost.com/2025/11/16/us-news/mtg-sugg...,48


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
47,55,2025-11-16-20-26-00 +0000,wsj,President Trump is making a last-ditch push to...,https://www.wsj.com/politics/policy/trump-play...
147,34,2025-11-16-01-42-19 +0000,cbc,Anti-government protesters clash with riot pol...,https://www.cbc.ca/news/world/gen-z-protest-me...
150,30,2025-11-16-00-29-01 +0000,bbc,My mum was a 17-year-old free spirit - so she ...,https://www.bbc.com/news/articles/cr43vx0rrwvo...
16,26,2025-11-16-22-45-28 +0000,nypost,Canal Street knock-off vendors back with venge...,https://nypost.com/2025/11/16/us-news/nyc-knoc...
49,26,2025-11-16-20-23-31 +0000,nypost,Jets cornerback Kris Boyd’s pal gives update o...,https://nypost.com/2025/11/16/sports/jets-corn...
100,25,2025-11-16-13-33-18 +0000,bbc,The US will help South Korea build nuclear 'at...,https://www.bbc.com/news/articles/c620qppzlgwo...
129,25,2025-11-16-05-06-13 +0000,nyt,4 Dead After Suspected Migrant Boat Overturns ...,https://www.nytimes.com/2025/11/15/us/panga-bo...
7,24,2025-11-16-23-33-06 +0000,nypost,Vile antisemitic graffiti scrawled on Mexico’s...,https://nypost.com/2025/11/16/world-news/antis...
80,24,2025-11-16-18-03-28 +0000,nypost,Schoolgirl murdered in her own bed by intruder...,https://nypost.com/2025/11/16/world-news/schoo...
11,21,2025-11-16-23-04-18 +0000,nypost,Video captures hero bystanders pulling injured...,https://nypost.com/2025/11/16/us-news/video-ca...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
